In [1]:
import json
import pandas as pd
from pandas.io.json import json_normalize
from utils import show_uniq_test_train
pd.set_option('display.max_colwidth', -1)

In [2]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
#drop non_useful
train.drop(["socialEngagementType"], axis=1, inplace=True)
test.drop(["socialEngagementType"], axis=1, inplace=True)
#convert data
train["date"] = pd.to_datetime(train.date.astype(str), format='%Y%m%d')
test["date"] = pd.to_datetime(test.date.astype(str), format='%Y%m%d')

/anaconda2/envs/env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#convert device json to dataframe
train_device = json_normalize(train.device.apply(json.loads))
test_device = json_normalize(test.device.apply(json.loads))
print("device")
print(str(train_device.columns == test_device.columns))
#convert geo to  dataframe
train_geo = json_normalize(train.geoNetwork.apply(json.loads))
test_geo = json_normalize(test.geoNetwork.apply(json.loads))
print("geo")
print(str(train_geo.columns == test_geo.columns))
#convert to trafficSource  dataframe
train_trafic = json_normalize(train.trafficSource.apply(json.loads))
test_trafic = json_normalize(test.trafficSource.apply(json.loads))
train_trafic.drop(["campaignCode"], axis=1, inplace=True)
print("traffic")
print(str(train_trafic.columns == test_trafic.columns))

device
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
geo
[ True  True  True  True  True  True  True  True  True  True  True]
traffic
[ True  True  True  True  True  True  True  True  True  True  True  True
  True]


In [4]:
#convert totals to  dataframe
train_totals = json_normalize(train.totals.apply(json.loads))
test_totals = json_normalize(test.totals.apply(json.loads))
target = train_totals.transactionRevenue
train_totals.drop(["transactionRevenue"], axis=1,inplace=True)
print(str(train_totals.columns == test_totals.columns))

[ True  True  True  True  True]


In [5]:
# drop json columns from original dataframe
train.drop(["totals", "trafficSource", "geoNetwork", "device"], axis=1, inplace=True)
test.drop(["totals", "trafficSource", "geoNetwork", "device"], axis=1, inplace=True)

In [6]:
#concatenate all sub df to one dataframe
train_main = pd.concat([train, train_device, train_geo, train_totals, train_trafic], axis=1)
test_main = pd.concat([test, test_device, test_geo, test_totals, test_trafic], axis=1)
train_main.columns== test_main.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [7]:
import gc
#remove unneeded  frames train, train_device, train_geo, train_totals, train_trafic
train, train_device, train_geo, train_totals, train_trafic = None, None, None, None, None
test, test_device, test_geo, test_totals, test_trafic = None, None, None, None, None
gc.collect()

21

In [8]:
# now we have dataframe without 
test_main.head()

,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,browser,browserSize,browserVersion,...,adwordsClickInfo.gclId,adwordsClickInfo.isVideoAd,adwordsClickInfo.page,adwordsClickInfo.slot,campaign,isTrueDirect,keyword,medium,referralPath,source
0,Organic Search,2017-10-16,6167871330617112363,6167871330617112363_1508151024,1508151024,2,1508151024,Chrome,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),True,(not provided),organic,NaN,google
1,Organic Search,2017-10-16,0643697640977915618,0643697640977915618_1508175522,1508175522,1,1508175522,Chrome,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
2,Organic Search,2017-10-16,6059383810968229466,6059383810968229466_1508143220,1508143220,1,1508143220,Chrome,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
3,Organic Search,2017-10-16,2376720078563423631,2376720078563423631_1508193530,1508193530,1,1508193530,Safari,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
4,Organic Search,2017-10-16,2314544520795440038,2314544520795440038_1508217442,1508217442,1,1508217442,Safari,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google


In [9]:
# analyze train vs test
show_uniq_test_train(train_main, test_main)

adContent ;train: [nan 'Full auto ad IMAGE ONLY' 'First Full Auto Template Test Ad'
 '{KeyWord:Google Brand Items}' 'Display Ad created 3/11/14'
 'Ad from 11/7/16' 'Display Ad created 3/11/15' 'Ad from 12/13/16'
 'Ad from 11/3/16' 'Google Online Store' 'Google Merchandise Collection'
 '{KeyWord:Want Google Stickers?}' '{KeyWord:Google Merchandise}'
 'Display Ad created 11/17/14' '{KeyWord:Google Drinkware}'
 "{KeyWord:Google Men's T-Shirts}" '{KeyWord:Google Branded Gear}'
 '{KeyWord:Google Branded Apparel}' 'Google Paraphernalia'
 'Swag with Google Logos' '{KeyWord:Google Branded Kit}'
 'Want Google Sunglasses' '{KeyWord:Looking for Google Bags?}'
 'Full auto ad TEXT ONLY' 'Google Merchandise' 'google store' 'LeEco_1a'
 'Full auto ad TEXT/NATIVE' '{KeyWord:Want Google Pet Toys?}'
 'Google Store' "Men's-Outerwear Google Apparel" '20% discount'
 'Full auto ad with Primary Color' 'JD_5a_v1'
 '{KeyWord:Google Branded Outerwear}' 'Full auto ad NATIVE ONLY'
 'url_builder' 'Google store' 'fr

cityId ;train: ['not available in demo dataset']; test:['not available in demo dataset']
continent ;train: ['Asia' 'Oceania' 'Europe' 'Americas' 'Africa' '(not set)']; test:['Asia' 'Europe' 'Americas' 'Africa' 'Oceania' '(not set)']
deviceCategory ;train: ['desktop' 'mobile' 'tablet']; test:['desktop' 'mobile' 'tablet']
flashVersion ;train: ['not available in demo dataset']; test:['not available in demo dataset']
isMobile ;train: [False  True]; test:[False  True]
isTrueDirect ;train: [nan True]; test:[True nan]
language ;train: ['not available in demo dataset']; test:['not available in demo dataset']
latitude ;train: ['not available in demo dataset']; test:['not available in demo dataset']
longitude ;train: ['not available in demo dataset']; test:['not available in demo dataset']
medium ;train: ['organic' 'referral' 'cpc' 'affiliate' '(none)' 'cpm' '(not set)']; test:['organic' 'cpc' 'cpm' '(none)' 'referral' 'affiliate' '(not set)']
metro ;train: ['(not set)' 'not available in demo da

In [10]:
# drop useless columns
useless = ["visits", "screenResolution", "screenColors", "operatingSystemVersion", "networkLocation", "mobileInputSelector",
          "mobileDeviceModel", "mobileDeviceMarketingName", "mobileDeviceInfo", "mobileDeviceBranding",
          "longitude", "latitude", "language", "flashVersion", "cityId", "browserVersion", "browserSize",
          "adwordsClickInfo.criteriaParameters"]
train_main.drop(useless, axis=1, inplace=True)
test_main.drop(useless, axis=1, inplace=True)

In [11]:
train_main.shape

(903653, 34)

In [12]:
test_main.shape

(804684, 34)

In [13]:
train_main.head()

,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,browser,deviceCategory,isMobile,...,adwordsClickInfo.gclId,adwordsClickInfo.isVideoAd,adwordsClickInfo.page,adwordsClickInfo.slot,campaign,isTrueDirect,keyword,medium,referralPath,source
0,Organic Search,2016-09-02,1131660440785968503,1131660440785968503_1472830385,1472830385,1,1472830385,Chrome,desktop,False,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
1,Organic Search,2016-09-02,377306020877927890,377306020877927890_1472880147,1472880147,1,1472880147,Firefox,desktop,False,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
2,Organic Search,2016-09-02,3895546263509774583,3895546263509774583_1472865386,1472865386,1,1472865386,Chrome,desktop,False,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
3,Organic Search,2016-09-02,4763447161404445595,4763447161404445595_1472881213,1472881213,1,1472881213,UC Browser,desktop,False,...,NaN,NaN,NaN,NaN,(not set),NaN,google + online,organic,NaN,google
4,Organic Search,2016-09-02,27294437909732085,27294437909732085_1472822600,1472822600,2,1472822600,Chrome,mobile,True,...,NaN,NaN,NaN,NaN,(not set),True,(not provided),organic,NaN,google


In [16]:
train_main[target.notnull()]

,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,browser,deviceCategory,isMobile,...,adwordsClickInfo.gclId,adwordsClickInfo.isVideoAd,adwordsClickInfo.page,adwordsClickInfo.slot,campaign,isTrueDirect,keyword,medium,referralPath,source
752,Direct,2016-09-02,6194193421514403509,6194193421514403509_1472843572,1472843572,1,1472843572,Chrome,desktop,False,...,NaN,NaN,NaN,NaN,(not set),True,NaN,(none),NaN,(direct)
753,Organic Search,2016-09-02,5327166854580374902,5327166854580374902_1472844906,1472844906,3,1472844906,Chrome,desktop,False,...,NaN,NaN,NaN,NaN,(not set),True,(not provided),organic,NaN,google
799,Referral,2016-09-02,8885051388942907862,8885051388942907862_1472827393,1472827393,7,1472827393,Chrome,desktop,False,...,NaN,NaN,NaN,NaN,(not set),True,NaN,referral,/,mall.googleplex.com
802,Referral,2016-09-02,0185467632009737931,0185467632009737931_1472846398,1472846398,6,1472846398,Chrome,desktop,False,...,NaN,NaN,NaN,NaN,(not set),True,NaN,referral,/,mall.googleplex.com
859,Referral,2016-09-02,3244885836845029978,3244885836845029978_1472824817,1472824817,4,1472824817,Chrome,desktop,False,...,NaN,NaN,NaN,NaN,(not set),True,NaN,referral,/,mall.googleplex.com
866,Referral,2016-09-02,3351538799616866750,3351538799616866750_1472855945,1472855945,2,1472855945,Chrome,desktop,False,...,NaN,NaN,NaN,NaN,(not set),True,NaN,referral,/,mall.googleplex.com
893,Direct,2016-09-02,1776658355119092313,1776658355119092313_1472861525,1472861525,1,1472861525,Safari,desktop,False,...,NaN,NaN,NaN,NaN,(not set),True,NaN,(none),NaN,(direct)
910,Referral,2016-09-02,770431600902969839,770431600902969839_1472836071,1472836071,13,1472836071,Chrome,desktop,False,...,NaN,NaN,NaN,NaN,(not set),True,NaN,referral,/,mall.googleplex.com
922,Organic Search,2016-09-02,7147112211830167925,7147112211830167925_1472853332,1472853332,1,1472853332,Chrome,mobile,True,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
925,Referral,2016-09-02,6664733704830724714,6664733704830724714_1472853995,1472853995,7,1472853995,Chrome,desktop,False,...,NaN,NaN,NaN,NaN,(not set),True,NaN,referral,/a/google.com/google-merchandise-store/home/on-site-store,sites.google.com


In [18]:
train_main[train_main.sessionId.duplicated()]

,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,browser,deviceCategory,isMobile,...,adwordsClickInfo.gclId,adwordsClickInfo.isVideoAd,adwordsClickInfo.page,adwordsClickInfo.slot,campaign,isTrueDirect,keyword,medium,referralPath,source
50181,Organic Search,2017-06-24,7980925080669177483,7980925080669177483_1498285182,1498285182,1,1498287677,Safari,tablet,True,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
78892,Paid Search,2017-06-16,996739878901151740,0996739878901151740_1497682650,1497682650,1,1497682650,Safari,tablet,True,...,EAIaIQobChMI4LPXyKbE1AIVElp-Ch3HpgroEAAYASAAEgKRX_D_BwE,False,1,Top,AW - Accessories,NaN,Google Merchandise,cpc,NaN,google
80071,Paid Search,2017-06-16,9107294226500038587,9107294226500038587_1497681738,1497681738,1,1497681738,Chrome,desktop,False,...,EAIaIQobChMI7PShnKPE1AIVj5x-Ch1rXQmUEAAYASAAEgLg-_D_BwE,False,1,Top,AW - Accessories,NaN,+Google +Swag,cpc,NaN,google
89617,Paid Search,2016-11-30,946085548949296102,946085548949296102_1480492063,1480492063,1,1480492882,Chrome,desktop,False,...,Cj0KEQiA6_TBBRDInaPjhcelt5oBEiQApPeTF8AaD2h3F01t0oDgcbc8EIpwCPRWxWQRAiewGMDZzHIaAiT-8P8HAQ,False,1,Top,AW - Dynamic Search Ads Whole Site,NaN,6qEhsCssdK0z36ri,cpc,NaN,google
89962,Organic Search,2016-11-30,7817946123013268752,7817946123013268752_1480492790,1480492790,1,1480492842,Chrome,desktop,False,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
91550,Social,2016-11-30,387871673225482446,0387871673225482446_1480492793,1480492793,1,1480492806,Safari,desktop,False,...,NaN,NaN,NaN,NaN,(not set),NaN,NaN,referral,/yt/about/,youtube.com
92145,Social,2016-11-30,1468326551501903507,1468326551501903507_1480492763,1480492763,1,1480492812,Safari,desktop,False,...,NaN,NaN,NaN,NaN,(not set),NaN,NaN,referral,/yt/about/ko/,youtube.com
118127,Organic Search,2017-04-21,8578907758685823541,8578907758685823541_1492757496,1492757496,1,1492758303,Chrome,desktop,False,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
119422,Organic Search,2017-04-21,2801742746913009915,2801742746913009915_1492757406,1492757406,1,1492758135,Chrome,mobile,True,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
135770,Direct,2017-01-15,4580211096974089358,4580211096974089358_1484552919,1484552919,1,1484552919,Chrome,desktop,False,...,NaN,NaN,NaN,NaN,(not set),True,NaN,(none),NaN,(direct)


In [ ]:
target[target.notnull()].hist(bins = 50)

KeyboardInterrupt: 